In [1]:
import torch
import torch.nn as nn

from framework import (
    SGD, MSE, ReLU, Sigmoid, Sequential,
    Conv2d, TransposeConv2d,
    )

/home/sepeehr/anaconda3/envs/DLC/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Test ReLU, Sigmoid, MSE, Sequential

In [2]:
x = torch.rand(5, 3, requires_grad=True) - .5
y = torch.rand_like(x, requires_grad=True) - .5

In [3]:
for func_, func in zip(
    [ReLU(), Sigmoid(), Sequential(ReLU(), Sigmoid())],
    [nn.ReLU(), nn.Sigmoid(), nn.Sequential(nn.ReLU(), nn.Sigmoid())],
):
    torch.testing.assert_allclose(func_(x), func(x))
    torch.testing.assert_allclose(
        func_.backward(grad=torch.ones_like(x)),
        torch.autograd.grad(func(x), x, grad_outputs=torch.ones_like(x))[0],
    )

func_, func = MSE(), nn.MSELoss()
torch.testing.assert_allclose(func_(x, y), func(x, y))
torch.testing.assert_allclose(
    func_.backward(grad=torch.ones_like(x))[0],
    torch.autograd.grad(func(x, y), (x, y))[0],
)
torch.testing.assert_allclose(
    func_.backward(grad=torch.ones_like(x))[1],
    torch.autograd.grad(func(x, y), (x, y))[1],
)

### Test Conv2d

In [4]:
in_channels, out_channels = 3, 32
settings = dict(
    kernel_size=2,
    stride=2,
    padding=0,
    dilation=1,
    )

# Make sure that the randomly generated parameters work fine
x = torch.randn((50, in_channels, 64, 64)).requires_grad_()
conv_gen = Conv2d(
    in_channels, out_channels, **settings,
    params=None,
    )
y_ = conv_gen(x)
_ = conv_gen.backward(grad=torch.ones_like(y_))

# Build the moduels and assert the parameters are identical
conv = nn.Conv2d(
    in_channels, out_channels, **settings
    )
conv_ = Conv2d(
    in_channels, out_channels, **settings,
    params=dict(weight=conv.weight.clone(), bias=conv.bias.clone())
    )
torch.testing.assert_allclose(conv_.param()[0][0], conv.weight)
torch.testing.assert_allclose(conv_.param()[1][0], conv.bias)

# Check the forward and backward methods
x = torch.randn((50, in_channels, 64, 64)).requires_grad_()
y, y_ = conv(x), conv_(x)
torch.testing.assert_allclose(y_, y)
torch.testing.assert_allclose(
        conv_.backward(grad=torch.ones_like(y_)),
        torch.autograd.grad(conv(x), x, grad_outputs=torch.ones_like(y))[0],
    )

# Check the parameter gradients
gradw, = torch.autograd.grad(conv(x), conv.weight, grad_outputs=torch.ones_like(y))
gradb, = torch.autograd.grad(conv(x), conv.bias, grad_outputs=torch.ones_like(y))
_, gradw_ = conv_.param()[0]
_, gradb_ = conv_.param()[1]
torch.testing.assert_allclose(gradw_, gradw)
torch.testing.assert_allclose(gradb_, gradb)

[W NNPACK.cpp:80] Could not initialize NNPACK! Reason: Unsupported hardware.


### Test TransposeConv2d

In [5]:
in_channels, out_channels = 3, 32
settings = dict(
    kernel_size=2,
    stride=4,
    padding=3,
    dilation=1,
    )

# Make sure that the randomly generated parameters work fine
x = torch.randn((50, in_channels, 64, 64)).requires_grad_()
conv_gen = TransposeConv2d(
    in_channels, out_channels, **settings,
    params=None,
    )
y_ = conv_gen(x)
_ = conv_gen.backward(grad=torch.ones_like(y_))

# Build the moduels and assert the parameters are identical
conv = nn.ConvTranspose2d(
    in_channels, out_channels, **settings
    )
conv_ = TransposeConv2d(
    in_channels, out_channels, **settings,
    params=dict(weight=conv.weight.clone(), bias=conv.bias.clone())
    )
torch.testing.assert_allclose(conv_.param()[0][0], conv.weight)
torch.testing.assert_allclose(conv_.param()[1][0], conv.bias)

# Check the forward and backward methods
x = torch.randn((50, in_channels, 64, 64)).requires_grad_()
y, y_ = conv(x), conv_(x)
torch.testing.assert_allclose(y_, y)
torch.testing.assert_allclose(
        conv_.backward(grad=torch.ones_like(y_)),
        torch.autograd.grad(conv(x), x, grad_outputs=torch.ones_like(y))[0],
    )

# Check the parameter gradients
gradw, = torch.autograd.grad(conv(x), conv.weight, grad_outputs=torch.ones_like(y))
gradb, = torch.autograd.grad(conv(x), conv.bias, grad_outputs=torch.ones_like(y))
_, gradw_ = conv_.param()[0]
_, gradb_ = conv_.param()[1]
torch.testing.assert_allclose(gradw_, gradw)
torch.testing.assert_allclose(gradb_, gradb)

### Test sequential CNN

In [6]:
x = torch.randn((50, 3, 28, 28)).requires_grad_()

# Build the convolutional layers
conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
conv1_ = Conv2d(in_channels=3, out_channels=32, kernel_size=3,
    params=dict(weight=conv1.weight.clone(), bias=conv1.bias.clone()))
conv2 = nn.Conv2d(in_channels=32, out_channels=8, kernel_size=5)
conv2_ = Conv2d(in_channels=32, out_channels=8, kernel_size=5,
    params=dict(weight=conv2.weight.clone(), bias=conv2.bias.clone()))

# Build the tansposed convolutional layers
tconv2 = nn.ConvTranspose2d(in_channels=8, out_channels=32, kernel_size=5)
tconv2_ = TransposeConv2d(in_channels=8, out_channels=32, kernel_size=5,
    params=dict(weight=tconv2.weight.clone(), bias=tconv2.bias.clone()))
tconv1 = nn.ConvTranspose2d(in_channels=32, out_channels=3, kernel_size=3)
tconv1_ = TransposeConv2d(in_channels=32, out_channels=3, kernel_size=3,
    params=dict(weight=tconv1.weight.clone(), bias=tconv1.bias.clone()))

# Build the sequences
seq = nn.Sequential(
    conv1,
    nn.ReLU(),
    conv2,
    nn.ReLU(),
    tconv2,
    nn.ReLU(),
    tconv1,
    nn.Sigmoid(),
)
seq_ = Sequential(
    conv1_,
    ReLU(),
    conv2_,
    ReLU(),
    tconv2_,
    ReLU(),
    tconv1_,
    Sigmoid(),
)

# Test the forward and backward methods
y, y_ = seq(x), seq_(x)
torch.testing.assert_allclose(y_, y)
torch.testing.assert_allclose(
        seq_.backward(grad=torch.ones_like(y_)),
        torch.autograd.grad(seq(x), x, grad_outputs=torch.ones_like(y))[0],
    )

Test gradient wrt the parameters:

In [13]:
x = torch.randn((50, 3, 28, 28)).requires_grad_()
y = torch.randn((50, 3, 28, 28))

# Define the loss functions
loss = nn.MSELoss()
loss_ = MSE()

# Check the forward method
torch.testing.assert_allclose(loss_(seq(x), y), loss(seq_(x), y),)

# Calculate the gradients wrt to the parameters (Calling the backward methods)
seq.zero_grad()
loss(seq_(x), y).backward()
seq_.zero_grad()
grad_inp, grad_tar = loss_.backward()
_ = seq_.backward(grad=grad_inp)

# Test the gradients wrt the parameters
rtol, atol = 1e-03, 5e-03
torch.testing.assert_allclose(conv1.weight.grad, conv1_.param()[0][1])
torch.testing.assert_allclose(conv1.bias.grad, conv1_.param()[1][1])
torch.testing.assert_allclose(conv2.weight.grad, conv2_.param()[0][1])
torch.testing.assert_allclose(conv2.bias.grad, conv2_.param()[1][1])
torch.testing.assert_allclose(tconv1.weight.grad, tconv1_.param()[0][1], rtol=rtol, atol=atol)
torch.testing.assert_allclose(tconv1.bias.grad, tconv1_.param()[1][1], rtol=rtol, atol=atol)
torch.testing.assert_allclose(tconv2.weight.grad, tconv2_.param()[0][1], rtol=rtol, atol=atol)
torch.testing.assert_allclose(tconv2.bias.grad, tconv2_.param()[1][1], rtol=rtol, atol=atol)